### Install the _Kubeflow-metadata_ library

In [1]:
# To use the latest publish `kubeflow-metadata` library, you can run:
!pip install kubeflow-metadata --user
# Install other packages:
!pip install pandas --user
# Then restart the Notebook kernel.

In [2]:
import pandas
from kubeflow.metadata import metadata
from datetime import datetime
from uuid import uuid4

In [ ]:
METADATA_STORE_HOST = "metadata-grpc-service.kubeflow" # default DNS of Kubeflow Metadata gRPC serivce.
METADATA_STORE_PORT = 8080

### Create a workspace

In [3]:
ws1 = metadata.Workspace(
    # Connect to metadata service in namesapce kubeflow in k8s cluster.
    store=metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=METADATA_STORE_PORT),
    name="workspace_1",
    description="a workspace for testing",
    labels={"n1": "v1"})

### Create a run in a workspace

In [4]:
r = metadata.Run(
    workspace=ws1,
    name="run-" + datetime.utcnow().isoformat("T") ,
    description="a run in ws_1",
)

### Create an execution in a run

In [5]:
exec = metadata.Execution(
    name = "execution" + datetime.utcnow().isoformat("T") ,
    workspace=ws1,
    run=r,
    description="execution example",
)
print("An execution is create with id %s" % exec.id)

An execution is create with id 290


### Log a data set

In [6]:
date_set_version = "data_set_version_" + str(uuid4())
data_set = exec.log_input(
        metadata.DataSet(
            description="an example data",
            name="mytable-dump",
            owner="owner@my-company.org",
            uri="file://path/to/dataset",
            version=date_set_version,
            query="SELECT * FROM mytable"))
print("data set id is {0.id} with version '{0.version}''".format(data_set))

data set id is 171 with version 'data_set_version_cbebc757-0d76-4e1e-bbd9-02b065e4c3ea''


### Log a model

In [7]:
model_version = "model_version_" + str(uuid4())
model = exec.log_output(
    metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
            hyperparameters={
                "learning_rate": 0.5,
                "layers": [10, 3, 1],
                "early_stop": True
            },
            version=model_version,
            labels={"mylabel": "l1"}))
print(model)
print("\nmodel id is {0.id} and version is {0.version}".format(model))

kubeflow.metadata.metadata.Model(workspace=None, name='MNIST', description='model to recognize handwritten digits', owner='someone@kubeflow.org', uri='gcs://my-bucket/mnist', version='model_version_50b419e2-af69-4c0e-a251-78246d4c0578', model_type='neural network', training_framework={'name': 'tensorflow', 'version': 'v1.0'}, hyperparameters={'learning_rate': 0.5, 'layers': [10, 3, 1], 'early_stop': True}, labels={'mylabel': 'l1'}, id=172, create_time='2019-12-04T00:44:49.444411Z', kwargs={})

model id is 172 and version is model_version_50b419e2-af69-4c0e-a251-78246d4c0578


### Log an evaluation of a model

In [8]:
metrics = exec.log_output(
    metadata.Metrics(
            name="MNIST-evaluation",
            description="validating the MNIST model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist-eval.csv",
            data_set_id=str(data_set.id),
            model_id=str(model.id),
            metrics_type=metadata.Metrics.VALIDATION,
            values={"accuracy": 0.95},
            labels={"mylabel": "l1"}))
print("metrics id is %s" % metrics.id)

metrics id is 173


### Add Metadata for serving the model

In [9]:
serving_application = metadata.Execution(
    name="serving model",
    workspace=ws1,
    description="an execution to represent model serving component",
)
# Noticed we use model name, version, uri to uniquely identify existing model.
served_model = metadata.Model(
    name="MNIST",
    uri="gcs://my-bucket/mnist",
    version=model.version,
)
m=serving_application.log_input(served_model)
print("Found the mode with id {0.id} and version '{0.version}'.".format(m))

Found the mode with id 172 and version 'model_version_50b419e2-af69-4c0e-a251-78246d4c0578'.


### List all models in the workspace

In [10]:
pandas.DataFrame.from_dict(ws1.list(metadata.Model.ARTIFACT_TYPE_NAME))

,id,workspace,run,create_time,version,owner,description,name,model_type,uri,training_framework,hyperparameters,labels,kwargs
0,152,workspace_1,run-2019-12-04T00:10:29.797245,2019-12-04T00:10:30.516009Z,model_version_68fd3ccd-bb61-496a-b6b3-b9a7ab7a...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
1,154,workspace_1,run-2019-12-04T00:10:29.797245,2019-12-04T00:14:44.401822Z,model_version_4757fb07-20f5-4443-8000-ba0ad5ca...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
2,156,workspace_1,run-2019-12-04T00:10:29.797245,2019-12-04T00:14:56.893195Z,model_version_aa162457-85e4-473d-9f20-8fe72637...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
3,160,workspace_1,run-2019-12-04T00:15:41.059380,2019-12-04T00:15:41.766812Z,model_version_7af7615c-ec8d-4e30-b765-14dd1d81...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
4,163,workspace_1,run-2019-12-04T00:43:16.177845,2019-12-04T00:43:16.907629Z,model_version_3d8540f1-f74f-4660-83dc-2b5a886d...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
5,166,workspace_1,run-2019-12-04T00:43:23.426877,2019-12-04T00:43:24.138877Z,model_version_9987cd2f-d354-49b5-955e-952f21ff...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
6,169,workspace_1,run-2019-12-04T00:43:47.136103,2019-12-04T00:43:47.851783Z,model_version_123141d7-a3ce-4ac4-ade9-e750ad18...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
7,172,workspace_1,run-2019-12-04T00:44:48.687078,2019-12-04T00:44:49.444411Z,model_version_50b419e2-af69-4c0e-a251-78246d4c...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}


### Basic Lineage Tracking

In [11]:
print("model id is %s\n" % model.id)
    
model_events = ws1.store.get_events_by_artifact_ids([model.id])

execution_ids = set(e.execution_id for e in model_events)
print("All executions related to the model are {}".format(execution_ids))
# assert execution_ids == set([serving_application.id, exec.id])

trainer_events = ws1.store.get_events_by_execution_ids([exec.id])
artifact_ids = set(e.artifact_id for e in trainer_events)
print("All artifacts related to the training event are {}".format(artifact_ids))# assert artifact_ids == set([model.id, metrics.id, data_set.id])

model id is 172

All executions related to the model are {290, 291}
All artifacts related to the training event are {171, 172, 173}
